In [29]:
%load_ext sql

In [30]:
%config SqlMagic.displaylimit = 15
# %config SqlMagic.displaylimit = None

In [ ]:
import datetime as dt
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text

In [42]:
sys_log_db = "system_data_log.db"

sql_engine = create_engine("sqlite:///%s" % sys_log_db, echo=False)

In [189]:
def create_voltage_table(force=False):
    with sql_engine.connect() as sql_conn:
        if force:
            sql_text = text("""DROP TABLE IF EXISTS voltages;
                            """)
            sql_conn.execute(sql_text)
        # define schema
        sql_text = text("""
            CREATE TABLE IF NOT EXISTS voltages (
                Timestamp TEXT,
                Vmain FLOAT,
                Vmain_raw FLOAT,
                Vaux FLOAT,
                Vaux_raw FLOAT,
                PRIMARY KEY (Timestamp)
                )
        """)
        sql_conn.execute(sql_text)
        sql_conn.commit()

In [ ]:
create_voltage_table()

In [193]:
def add_voltage_log(Vmain, Vmain_raw, Vaux, Vaux_raw):
    with sql_engine.connect() as sql_conn:
        sql_text = text("""INSERT INTO voltages
                           VALUES (DateTime("now", "localtime"),
                                   :v_main,
                                   :v_main_raw,
                                   :v_aux,
                                   :v_aux_raw
                           );
                        """)
        sql_conn.execute(sql_text, {"v_main": Vmain,
                                    "v_main_raw": Vmain_raw,
                                    "v_aux": Vaux,
                                    "v_aux_raw": Vaux_raw})
        sql_conn.commit()

In [194]:
add_voltage_log(12.4, 12.8, 12.5, 12)

In [195]:
add_voltage_log(12.5, 12.75, 12.4, 11.9)

---

In [282]:
def get_voltages(voltage_type="Vmain", trailing_seconds=None):
    with sql_engine.connect() as sql_conn:
        sql_text = text(f"""SELECT Timestamp, {voltage_type}
                            FROM voltages
                         """)
        result_df = pd.read_sql(sql_text, con=sql_conn, index_col = "Timestamp", parse_dates=["Timestamp"])
    if trailing_seconds is not None:
        return result_df[result_df.index > dt.datetime.now() - dt.timedelta(seconds=trailing_seconds)]
    else:
        return result_df

In [343]:
def get_voltages(voltage_type=None, trailing_seconds=None):
    if voltage_type is None:
        cols = "*"
    else:
        cols = "Timestamp, %s" % voltage_type

    if trailing_seconds is not None:
        timestamp_now = dt.datetime.now()
        timestamp_trail = timestamp_now - dt.timedelta(seconds=trailing_seconds)
        timestamp_trail_str = timestamp_trail.strftime("%Y-%m-%d %H:%M:%S")
        time_filter = "WHERE Timestamp > %s" % timestamp_trail_str
    else:
        time_filter = ""

    with sql_engine.connect() as sql_conn:
        # sql_text = text(f"""SELECT Timestamp, {voltage_type}
        sql_text = text(f"""SELECT {cols}
                            FROM voltages
                            {time_filter}
                         """)
        result_df = pd.read_sql(sql_text, con=sql_conn, index_col = "Timestamp", parse_dates=["Timestamp"])
    return result_df

In [344]:
df = get_voltages("Vmain"); df

,Vmain
Timestamp,
2025-10-12 15:08:14,12.4
2025-10-12 15:08:21,12.4
2025-10-12 15:08:22,12.5


---

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
pandas.DataFrame.to_sql(sys_log_db, )